# Preparation

Preparation biasa dilakukan untuk mempersiapkan data sebelum masuk dalam tahap pemodelan. <br>
Berikut adalah tahapan yang akan dilalui pada data `SC_HW1_bank_data.csv` sebelum tahap pemodelan :
1. Import Library
2. Input Dataset
3. Preprocessing
4. Train-Test Split

## Import Library

In [1]:
import pandas as pd
import numpy as np

## Input Dataset

In [2]:
#Membaca data dan memasukkannya ke dalam bentuk Pandas Dataframe
df = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')

In [3]:
#Jalankan code untuk mengecek nama kolom yang tersedia
df.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
#Hilangkan kolom yang dirasa tidak relevan dengan model (contoh: ID). None dapat diisi dengan nama-nama kolom yang akan digunakan.
#Contoh df = df[['X1','X2', 'Y']].copy()

#START CODE
df = df[['Geography', 'Gender', 'Age', 'Tenure','Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary', 'Exited']].copy()

## Preprocessing

In [5]:
#Lakukan One-Hot Encoder untuk data categorical, dengan fungsi pandas get_dummies

#START CODE
df = pd.DataFrame(df)
df_encoded = pd.get_dummies(df, columns=['Geography', 'Gender'])

In [6]:
#Pisahkan mana X (feature) dengan Y,
#Y adalah kolom "Exited"

#START CODE
X = df_encoded.drop(columns=['Exited'])
y = df_encoded['Exited']

In [7]:
#Lakukan Scaler dan/atau Noermalisasi jika diperlukan
from sklearn.preprocessing import MinMaxScaler

#START CODE
scaler = MinMaxScaler()
X_transform = scaler.fit_transform(X)


In [8]:
#Ini digunakan jika dilakukan scaler/Normalisas. Jika tidak, code ini bisa dilewat dan diganti dengan code yang ada di dalam komen
X_transform = pd.DataFrame(X_transform, columns = X.columns)
#X_transform = X.copy()

## Train-Test Split

In [9]:
#Split menjadi train dan test dengan test_size 25%
#Tidak perlu mengubah code ini

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_transform,y,test_size = 0.25,random_state = 123)

# Modeling

## Model1

### Soal :
Jelaskan secara Singkat Model pertama yang digunakan!

Random Forest adalah model ensambel yang terdiri dari beberapa pohon keputusan. Setiap pohon keputusan dibangun secara independen, dan hasil prediksi diambil dengan mengumpulkan hasil dari semua pohon. Random Forest efektif dalam mengatasi overfitting, mampu menangani data berdimensi tinggi, dan dapat digunakan untuk klasifikasi dan regresi.

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [11]:
#START CODE
model1 = RandomForestClassifier()
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model1,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'max_depth': None,
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 50}

In [12]:
#lakukan evaluasi dengan beberapa maetric di bawah ini
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)


Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1983
           1       0.78      0.46      0.58       517

    accuracy                           0.86      2500
   macro avg       0.83      0.71      0.75      2500
weighted avg       0.85      0.86      0.85      2500

Confusion Matrix:
[[1917   66]
 [ 279  238]]
Akurasi: 0.862


## Model2
### Soal :
Jelaskan secara Singkat Model ke-2 yang digunakan!

Support Vector Machine (SVM) adalah model pembelajaran mesin yang digunakan untuk klasifikasi dan regresi. SVM berfokus pada pemisahan dua kelas dengan mencari "margin" terbesar antara kedua kelas. SVM dapat menangani data linier dan non-linier dengan menggunakan kernel, seperti kernel radial (RBF) atau kernel polinomial. SVM dikenal dengan kemampuannya dalam menangani data yang tidak terdistribusi secara merata dan memiliki tingkat akurasi yang tinggi.

In [14]:
#START CODE
model2 = SVC()
params2 = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1]
}

# Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
grid2 = GridSearchCV(
    estimator=model2,
    param_grid=params2,
    scoring='accuracy',
    n_jobs=10,
    cv=10
)

grid2.fit(X_train, y_train)
grid2.best_params_

{'C': 10, 'gamma': 1, 'kernel': 'poly'}

In [16]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid2.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      1983
           1       0.82      0.44      0.57       517

    accuracy                           0.86      2500
   macro avg       0.85      0.71      0.75      2500
weighted avg       0.86      0.86      0.85      2500

Confusion Matrix:
[[1934   49]
 [ 289  228]]
Akurasi: 0.8648


## Model3
### Soal :
Jelaskan secara Singkat Model ke-3 yang digunakan!

Logistic Regression (Regresi Logistik) adalah model statistik yang digunakan untuk masalah klasifikasi. Meskipun namanya mengandung kata "regresi," ini adalah algoritma klasifikasi. Logistic Regression memodelkan probabilitas bahwa sampel termasuk dalam suatu kelas dengan menggunakan fungsi logistik. Model ini cocok untuk tugas klasifikasi biner dan multikelas, dan relatif mudah untuk diinterpretasi. Ini adalah model linier yang cocok untuk pemisahan linier antara kelas, meskipun ekstensi seperti regresi logistik multinomial dapat digunakan untuk masalah multikelas.

In [20]:
#START CODE
model3 = LogisticRegression()
params3 = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

# Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
grid3 = GridSearchCV(
    estimator=model3,
    param_grid=params3,
    scoring='accuracy',
    n_jobs=10,
    cv=10
)

grid3.fit(X_train, y_train)
grid3.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

{'C': 0.1, 'penalty': 'l2'}

In [22]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid3.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)


Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89      1983
           1       0.72      0.15      0.25       517

    accuracy                           0.81      2500
   macro avg       0.77      0.57      0.57      2500
weighted avg       0.80      0.81      0.76      2500

Confusion Matrix:
[[1953   30]
 [ 439   78]]
Akurasi: 0.8124


## Tarik Kesimpulan Model Mana yang terbaik beserta alasannya

Kesimpulannya model terbaik untuk mendapat akurasi tertinggi adalah SVM dengan akurasi 86,4%. Alasannya ada 4 yaitu
1. Kemampuan untuk Menangani Data Non-linear: SVM memiliki kemampuan bawaan untuk menangani pemisahan kelas yang kompleks, terutama dengan menggunakan kernel seperti 'rbf' atau 'poly'. Ini memungkinkan SVM untuk berperforma baik pada dataset yang tidak memiliki pemisahan linier yang jelas.

2. Optimasi Terbaik: SVM berusaha untuk menemukan margin maksimum antara kelas yang memungkinkan model memisahkan data dengan baik. Oleh karena itu, SVM biasanya memberikan pemisahan kelas yang lebih baik dibandingkan dengan beberapa model lainnya.

3. Pengendalian Overfitting: SVM memiliki parameter seperti C yang memungkinkan pengendalian ketat terhadap overfitting. Dengan penalaian yang tepat terhadap parameter-parameter ini, Anda dapat menghindari overfitting pada model SVM Anda.

4. Dukungan Vektor: SVM hanya bergantung pada sejumlah kecil "dukungan vektor" untuk membuat keputusan klasifikasi. Hal ini memungkinkan SVM untuk lebih tahan terhadap outliers dan noise dalam data.